# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4fd8164ee0>
├── 'a' --> tensor([[ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645]])
└── 'x' --> <FastTreeValue 0x7f4fdd1efbe0>
    └── 'c' --> tensor([[ 0.3764,  0.3948, -0.2520, -0.3134],
                        [ 0.6965,  0.4104, -0.0521,  0.5239],
                        [ 2.2649,  1.4015,  0.6586,  1.4763]])

In [4]:
t.a

tensor([[ 2.0230, -0.5560,  1.1181],
        [ 0.1663,  0.3035, -0.1645]])

In [5]:
%timeit t.a

61.5 ns ± 0.0903 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4fd8164ee0>
├── 'a' --> tensor([[ 0.3974, -1.4951,  0.3751],
│                   [-1.3653,  1.6876,  0.3010]])
└── 'x' --> <FastTreeValue 0x7f4fdd1efbe0>
    └── 'c' --> tensor([[ 0.3764,  0.3948, -0.2520, -0.3134],
                        [ 0.6965,  0.4104, -0.0521,  0.5239],
                        [ 2.2649,  1.4015,  0.6586,  1.4763]])

In [7]:
%timeit t.a = new_value

59.6 ns ± 0.0199 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645]]),
    x: Batch(
           c: tensor([[ 0.3764,  0.3948, -0.2520, -0.3134],
                      [ 0.6965,  0.4104, -0.0521,  0.5239],
                      [ 2.2649,  1.4015,  0.6586,  1.4763]]),
       ),
)

In [10]:
b.a

tensor([[ 2.0230, -0.5560,  1.1181],
        [ 0.1663,  0.3035, -0.1645]])

In [11]:
%timeit b.a

51.3 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1015, -0.0649, -1.7227],
               [ 0.1771, -0.5924, -1.7783]]),
    x: Batch(
           c: tensor([[ 0.3764,  0.3948, -0.2520, -0.3134],
                      [ 0.6965,  0.4104, -0.0521,  0.5239],
                      [ 2.2649,  1.4015,  0.6586,  1.4763]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.305 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

859 ns ± 5.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 46.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 344 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4fdd1d8340>
├── 'a' --> tensor([[[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]],
│           
│                   [[ 2.0230, -0.5560,  1.1181],
│                    [ 0.1663,  0.3035, -0.1645]]])
└── 'x' --> <FastTreeValue 0x7f4fd816afa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 65.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4f34ce2610>
├── 'a' --> tensor([[ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645],
│                   [ 2.0230, -0.5560,  1.1181],
│                   [ 0.1663,  0.3035, -0.1645]])
└── 'x' --> <FastTreeValue 0x7f4fdb1ac490>
    └── 'c' --> tensor([[ 0.3764,  0.3948, -0.2520, -0.3134],
                        [ 0.6965,  0.4104, -0.0521,  0.5239],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 67.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]],
       
               [[ 2.0230, -0.5560,  1.1181],
                [ 0.1663,  0.3035, -0.1645]]]),
    x: Batch(
           c: tensor([[[ 0.3764,  0.3948, -0.2520, -0.3134],
                       [ 0.6965,  0.4104, -0.0521,  0.5239],
                       [ 2.2649,  1.4015,  0.6586,  1.4763]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645],
               [ 2.0230, -0.5560,  1.1181],
               [ 0.1663,  0.3035, -0.1645]]),
    x: Batch(
           c: tensor([[ 0.3764,  0.3948, -0.2520, -0.3134],
                      [ 0.6965,  0.4104, -0.0521,  0.5239],
                      [ 2.2649,  1.4015,  0.6586,  1.4763],
                      [ 0.3764,  0.3948, -0.2520, -0.3134],
                      [ 0.6965,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

340 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
